In [1]:
############## PLEASE RUN THIS CELL FIRST! ###################

# import everything and define a test runner function
from importlib import reload
from helper import run
import ecc
import helper
import tx
import script

In [2]:
from tx import Tx
from io import BytesIO
raw_tx = ('0100000001813f79011acb80925dfe69b3def355fe914bd1d96a3f5f71bf8303c6a989c7d1000000006b483045022100ed81ff192e75a3fd2304004dcadb746fa5e24c5031ccfcf21320b0277457c98f02207a986d955c6e0cb35d446a89d3f56100f4d7f67801c31967743a9c8e10615bed01210349fc4e631e3624a545de3f89f5d8684c7b8138bd94bdd531d2e213bf016b278afeffffff02a135ef01000000001976a914bc3b654dca7e56b04dca18f2566cdaf02e8d9ada88ac99c39800000000001976a9141c4bc762dd5423e332166702cb75f40df79fea1288ac19430600')
stream = BytesIO(bytes.fromhex(raw_tx))
transaction = Tx.parse(stream)
print(transaction.fee() >= 0)

True


In [3]:
from ecc import S256Point, Signature
sec = bytes.fromhex('0349fc4e631e3624a545de3f89f5d8684c7b8138bd94bdd531d2e213bf016b278a')
der = bytes.fromhex('3045022100ed81ff192e75a3fd2304004dcadb746fa5e24c5031ccfcf21320b0277457c98f02207a986d955c6e0cb35d446a89d3f56100f4d7f67801c31967743a9c8e10615bed')
z = 0x27e0c5994dec7824e56dec6b2fcb342eb7cdb0d0957c2fce9882f715e85d81a6
point = S256Point.parse(sec)
signature = Signature.parse(der)
print(point.verify(z, signature))

True


In [4]:
from helper import hash256
modified_tx = bytes.fromhex('0100000001813f79011acb80925dfe69b3def355fe914bd1d96a3f5f71bf8303c6a989c7d1000000001976a914a802fc56c704ce87c42d7c92eb75e7896bdc41ae88acfeffffff02a135ef01000000001976a914bc3b654dca7e56b04dca18f2566cdaf02e8d9ada88ac99c39800000000001976a9141c4bc762dd5423e332166702cb75f40df79fea1288ac1943060001000000')
h256 = hash256(modified_tx)
z = int.from_bytes(h256, 'big')
print(hex(z))

0x27e0c5994dec7824e56dec6b2fcb342eb7cdb0d0957c2fce9882f715e85d81a6


In [5]:
from ecc import S256Point, Signature
sec = bytes.fromhex('0349fc4e631e3624a545de3f89f5d8684c7b8138bd94bdd531d2e213bf016b278a')
der = bytes.fromhex('3045022100ed81ff192e75a3fd2304004dcadb746fa5e24c5031ccfcf21320b0277457c98f02207a986d955c6e0cb35d446a89d3f56100f4d7f67801c31967743a9c8e10615bed')
z = 0x27e0c5994dec7824e56dec6b2fcb342eb7cdb0d0957c2fce9882f715e85d81a6
point = S256Point.parse(sec)
signature = Signature.parse(der)
point.verify(z, signature)

True

### Exercise 1

Write the `sig_hash` method for the `Tx` class.

#### Make [this test](/edit/code-ch07/tx.py) pass: `tx.py:TxTest:test_sig_hash`

In [16]:
# Exercise 1

reload(tx)
run(tx.TxTest("test_sig_hash"))

.
----------------------------------------------------------------------
Ran 1 test in 0.005s

OK


### Exercise 2

Write the `verify_input` method for the `Tx` class. You will want to use the `TxIn.script_pubkey()`, `Tx.sig_hash()` and `Script.evaluate()` methods.

#### Make [this test](/edit/code-ch07/tx.py) pass: `tx.py:TxTest:test_verify_p2pkh`

In [18]:
# Exercise 2

reload(tx)
run(tx.TxTest("test_verify_p2pkh"))

.
----------------------------------------------------------------------
Ran 1 test in 0.280s

OK


In [19]:
from helper import decode_base58, SIGHASH_ALL
from script import p2pkh_script, Script
from tx import TxIn, TxOut, Tx
prev_tx = bytes.fromhex('0d6fe5213c0b3291f208cba8bfb59b7476dffacc4e5cb66f6eb20a080843a299')
prev_index = 13
tx_in = TxIn(prev_tx, prev_index)
tx_outs = []
change_amount = int(0.33*100000000)
change_h160 = decode_base58('mzx5YhAH9kNHtcN481u6WkjeHjYtVeKVh2')
change_script = p2pkh_script(change_h160)
change_output = TxOut(amount=change_amount, script_pubkey=change_script)
target_amount = int(0.1*100000000)
target_h160 = decode_base58('mnrVtF8DWjMu839VW3rBfgYaAfKk8983Xf')
target_script = p2pkh_script(target_h160)
target_output = TxOut(amount=target_amount, script_pubkey=target_script)
tx_obj = Tx(1, [tx_in], [change_output, target_output], 0, True)
print(tx_obj)

tx: cd30a8da777d28ef0e61efe68a9f7c559c1d3e5bcd7b265c850ccb4068598d11
version: 1
tx_ins:
0d6fe5213c0b3291f208cba8bfb59b7476dffacc4e5cb66f6eb20a080843a299:13
tx_outs:
33000000:OP_DUP OP_HASH160 d52ad7ca9b3d096a38e752c2018e6fbc40cdf26f OP_EQUALVERIFY OP_CHECKSIG
10000000:OP_DUP OP_HASH160 507b27411ccf7f16f10297de6cef3f291623eddf OP_EQUALVERIFY OP_CHECKSIG
locktime: 0


In [20]:
from ecc import PrivateKey
from helper import SIGHASH_ALL
z = transaction.sig_hash(0)
private_key = PrivateKey(secret=8675309)
der = private_key.sign(z).der()
sig = der + SIGHASH_ALL.to_bytes(1, 'big')
sec = private_key.point.sec()
script_sig = Script([sig, sec])
transaction.tx_ins[0].script_sig = script_sig 
print(transaction.serialize().hex())

NotImplementedError: 

In [23]:
from ecc import PrivateKey
from helper import hash256, little_endian_to_int
secret = little_endian_to_int(hash256(b'Jimmy Song secret'))
private_key = PrivateKey(secret)
print(private_key.point.address(testnet=True))

mn81594PzKZa9K3Jyy1ushpuEzrnTnxhVg


### Exercise 3

Write the `sign_input` method for the `Tx` class.

#### Make [this test](/edit/code-ch07/tx.py) pass: `tx.py:TxTest:test_sign_input`

In [29]:
# Exercise 3

reload(tx)
run(tx.TxTest("test_sign_input"))

.
----------------------------------------------------------------------
Ran 1 test in 0.235s

OK


### Exercise 4

Create a testnet transaction that sends 60% of a single UTXO to `mwJn1YPMq7y5F8J3LkC5Hxg9PHyZ5K4cFv`. The remaining amount minus fees should go back to your own change address. This should be a 1 input, 2 output transaction.

You can broadcast the transaction at https://testnet.blockchain.info/pushtx

In [30]:
# Exercise 4

from ecc import PrivateKey
from helper import decode_base58, SIGHASH_ALL, hash256, little_endian_to_int
from script import p2pkh_script, Script
from tx import TxIn, TxOut, Tx

# create 1 TxIn and 2 TxOuts
# 1 of the TxOuts should be back to your address
# the other TxOut should be to this address
target_address = 'mwJn1YPMq7y5F8J3LkC5Hxg9PHyZ5K4cFv'
utxo_amount = 0.0001
# target amount should be 60% of the output amount
target_amount = utxo_amount*0.6

# get the private key from the exercise in Chapter 4
# change address should be the address generated from Chapter 4
passphrase = b'' # get this from local file 'testnet-passphrase.txt'
secret = little_endian_to_int(hash256(passphrase))
priv_key = PrivateKey(secret)
change_address = priv_key.point.address(compressed=True,testnet=True)
# change amount = amount from the prev tx - target amount - fee
change_amount = (utxo_amount*0.3) # fee = 10%, change = 30%

# get the prev_tx and prev_index from the transaction where you got some testnet coins
prev_tx = bytes.fromhex('33bd46cda1486441447a274f4d5f1d9ff2acf7188301fef20b368dc8d4088514')
prev_index = 0
tx_ins = []
tx_ins.append(TxIn(prev_tx,prev_index))
tx_outs = []

# create a transaction input for the previous transaction with the default ScriptSig and sequence
h160 = decode_base58(target_address)
script_pubkey = p2pkh_script(h160)
target_satoshis = int(target_amount*100000000)
tx_outs.append(TxOut(target_satoshis, script_pubkey))
h160 = decode_base58(change_address)
script_pubkey = p2pkh_script(h160)
change_satoshis = int(change_amount*100000000)
tx_outs.append(TxOut(change_satoshis, script_pubkey))
tx_obj = Tx(1, tx_ins, tx_outs, 0, testnet=True)
print(tx_obj.sign_input(0, priv_key))
print(tx_obj.serialize().hex())

# create a transaction output for the target amount and address
# create a transaction output for the change amount and address
# create the transaction object

# sign the one input in the transaction object using the private key
# print the transaction's serialization in hex
# broadcast at http://testnet.blockchain.info/pushtx

True
0100000001148508d4c88d360bf2fe018318f7acf29f1d5f4d4f277a44416448a1cd46bd33000000006a4730440220790627161b93e4c46fffd0c8c9666676915c14a330908e614034bcccb6c4ea7b0220273e53c7586cc7510ec8732a47cd8936f28b25244260db3de3542007078449b5012102f84ccc50f407e61ec0088cd6ceca8aa4dbc704033159f49a8f339a7a3c2328baffffffff0270170000000000001976a914ad346f8eb57dee9a37981716e498120ae80e44f788acb80b0000000000001976a914910369be8e99c0a842ff3499b6bc13041c2ccf1088ac00000000


### Exercise 5

Advanced: get some more testnet coins from a testnet faucet and create a 2 input, 1 output transaction. 1 input should be from the faucet, the other should be from the previous exercise, the output can be your own address.

You can broadcast the transaction at https://testnet.blockchain.info/pushtx

In [33]:
# Exercise 5

from ecc import PrivateKey
from helper import decode_base58, SIGHASH_ALL
from script import p2pkh_script, Script
from tx import TxIn, TxOut, Tx

# Create 2 TxIns, 1 from the Exercise 4 and 1 from a testnet faucet
# Creat 1 TxOut to the address above

# get the private key from the exercise in Chapter 4
passphrase = b'' # get this from local file 'testnet-passphrase.txt
secret = little_endian_to_int(hash256(passphrase))
priv_key = PrivateKey(secret)
output_address = priv_key.point.address(compressed=True,testnet=True)

tx_ins = []
tx_outs = []
# get the prev_tx and prev_index from the transaction where you got some testnet coins
# create the first transaction input with the default ScriptSig and sequence
prev_tx_1 = bytes.fromhex('02f947bcd68306e08244d60081cdd9a311f1bcc58ecf21075192dfb40d9cb068')
prev_idx_1 = 1
tx1 = TxIn(prev_tx_1,prev_idx_1)
tx_ins.append(tx1)

# get the prev_tx and prev_index from the transaction in Exercise 4
# create the second transaction input with the default ScriptSig and sequence
prev_tx_2 = bytes.fromhex('cea2c9ee68a86ad5990456fba46df72b75a002cf8baa6c35e7b41af0988da5e4')
prev_idx_2 = 0
tx2 = TxIn(prev_tx_2,prev_idx_2)
tx_ins.append(tx2)

# set the fee to some reasonable amount
# target amount should be the sum of the inputs - fee
target_amount = 0.00013 * 0.9 # 10% fee
h160 = decode_base58(output_address)
script_pubkey = p2pkh_script(h160)
target_satoshis = int(target_amount*100000000)

# create a transaction output for the amount and address
tx_outs = []
tx_outs.append(TxOut(target_satoshis, script_pubkey))
tx_obj = Tx(1, tx_ins, tx_outs, 0, testnet=True)

# sign the first input using the private key
print(tx_obj.sign_input(0, priv_key))
# sign the second input using the private key 
print(tx_obj.sign_input(1, priv_key))
# print the transaction's serialization in hex
print(tx_obj.serialize().hex())

# broadcast at http://testnet.blockchain.info/pushtx

True
True
010000000268b09c0db4df92510721cf8ec5bcf111a3d9cd8100d64482e00683d6bc47f902010000006a47304402205f22babc15088d35efa819166d9c6dfaf3792f38bf2d7ab9057a0c1831ea721102205b9741c4c9e06187e1fffd494eb06eaf1062bdee3ab83c33995b4e1eea86384a012102f84ccc50f407e61ec0088cd6ceca8aa4dbc704033159f49a8f339a7a3c2328baffffffffe4a58d98f01ab4e7356caa8bcf02a0752bf76da4fb560499d56aa868eec9a2ce000000006a47304402202b3d85fc1136b50b9fcfba2f81addfb2d1de94dc715d552f84888c2ae6ab665c02201999c782e9286345a5802e708e9c370f78ae8dec1563fbfc130b56f11eea95ac012102f84ccc50f407e61ec0088cd6ceca8aa4dbc704033159f49a8f339a7a3c2328baffffffff01b42d0000000000001976a914910369be8e99c0a842ff3499b6bc13041c2ccf1088ac00000000
